# Compare 202405-2xrez to 202405 Bathymetry - Tile Rows 16 to 22

Detailed, tile by tile, comparison of 202405-2xrez and 202405 bathymetries including preliminary river files.
This notebook is used iteratively with the `Process202405-2xrezBathymetry.ipynb` notebook to identify features
in the 2xrez bathymetry that need to be adjusted in the latter notebook.

This notebook is based on the ones that Susan created to do the same thing during the creation of the 202108
bathymetry (which was known as 202103d at the time):
* `analysis-susan/notebooks/bathymetry/lookat201702_201803d_andrivers.ipynb`
* `analysis-susan/notebooks/bathymetry/lookat201702_201803d_middle.ipynb`
* `analysis-susan/notebooks/bathymetry/lookat201702_201803d_upper.ipynb`

The conda environment file for this notebook is `environment-2xrez.yaml`.

In [1]:
from pathlib import Path
import sys

import cmocean.cm as cm
import matplotlib.pyplot as plt
import numpy
import xarray

from plot_tile import plot_tile
from salishsea_tools import viz_tools

Python and library versions:

In [2]:
import bottleneck
import cmocean
import h5netcdf
import h5py
import matplotlib
import netCDF4
import numpy
import pandas
import scipy
import xarray

print(f"Python {sys.version=}")
print(f"{numpy.__version__=}")
print(f"{xarray.__version__=}")
print(f"{bottleneck.__version__=}")
print(f"{pandas.__version__=}")
print(f"{h5netcdf.__version__=}")
print(f"{h5py.__version__=}")
print(f"{netCDF4.__version__=}")
print(f"{scipy.__version__=}")
print(f"{matplotlib.__version__=}")
print(f"{cmocean.__version__=}")

Python sys.version='3.12.3 | packaged by conda-forge | (main, Apr 15 2024, 18:38:13) [GCC 12.3.0]'
numpy.__version__='1.26.4'
xarray.__version__='2024.6.0'
bottleneck.__version__='1.4.0'
pandas.__version__='2.2.2'
h5netcdf.__version__='1.3.0'
h5py.__version__='3.11.0'
netCDF4.__version__='1.6.5'
scipy.__version__='1.14.0'
matplotlib.__version__='3.8.4'
cmocean.__version__='v3.0.3'


## Load Bathymetry Datasets

In [3]:
# isolate the probable user-specific part of repo paths to make the notebook portable
repos_prefix = Path("/media/doug/warehouse/MEOPAR/")
repos_prefix = repos_prefix if repos_prefix.exists() else Path("/data/dlatorne/MEOPAR/")
# repos_prefix = Path("/home/sallen/MEOPAR/")

In [4]:
bathy_paths = {
    "final_202405": repos_prefix / "grid" / "bathymetry_202405.nc",
    "dbl_202405_base": repos_prefix / "grid" / "bathymetry_double_202405_base.nc",
    "dbl_202405": repos_prefix / "grid" / "bathymetry_double_202405.nc",
}

In [5]:
with xarray.open_dataset(bathy_paths["final_202405"], engine="h5netcdf") as bathy_ds:
    bathy = bathy_ds.Bathymetry
    lons = bathy_ds.nav_lon
    lats = bathy_ds.nav_lat

with xarray.open_dataset(bathy_paths["dbl_202405_base"], engine="h5netcdf") as dbl_bathy_base_ds:
    dbl_bathy_base_unmasked = dbl_bathy_base_ds.Bathymetry
    dbl_bathy_base = dbl_bathy_base_unmasked.where(dbl_bathy_base_unmasked != 0, numpy.nan)
    lons_dbl = dbl_bathy_base_ds.nav_lon
    lats_dbl = dbl_bathy_base_ds.nav_lat

with xarray.open_dataset(bathy_paths["dbl_202405"], engine="h5netcdf") as dbl_bathy_ds:
    dbl_bathy = dbl_bathy_ds.Bathymetry

Calculate expanded `lons` and `lats` arrays by extrapolating values on south and east edges.
This provides arrays that `pcoulormesh()` interprets as cell edges so that it doesn't a
`UserWarning` about the non-montonic lon/lat values that arise due to the rotated grid.
This code is based on Michael's `expandf()` function in
https://github.com/SalishSeaCast/analysis-michael/blob/master/bathymetry/bathy_helpers.py

In [6]:
lons_e = numpy.zeros((lons.shape[0]+1, lons.shape[1]+1))
lons_e[1:, 1:] = lons
lons_e[0, 1:] = lons[0, :] - (lons[1, :] - lons[0, :])
lons_e[:, 0] = lons_e[:, 1] - (lons_e[:, 2] - lons_e[:, 1])

lats_e = numpy.zeros((lats.shape[0]+1, lats.shape[1]+1))
lats_e[1:, 1:] = lats
lats_e[0, 1:] = lats[0, :] - (lats[1, :] - lats[0, :])
lats_e[:, 0] = lats_e[:, 1] - (lats_e[:, 2] - lats_e[:, 1])

lons_dbl_e = numpy.zeros((lons_dbl.shape[0]+1, lons_dbl.shape[1]+1))
lons_dbl_e[1:, 1:] = lons_dbl
lons_dbl_e[0, 1:] = lons_dbl[0, :] - (lons_dbl[1, :] - lons_dbl[0, :])
lons_dbl_e[:, 0] = lons_dbl_e[:, 1] - (lons_dbl_e[:, 2] - lons_dbl_e[:, 1])

lats_dbl_e = numpy.zeros((lats_dbl.shape[0]+1, lats_dbl.shape[1]+1))
lats_dbl_e[1:, 1:] = lats_dbl
lats_dbl_e[0, 1:] = lats_dbl[0, :] - (lats_dbl[1, :] - lats_dbl[0, :])
lats_dbl_e[:, 0] = lats_dbl_e[:, 1] - (lats_dbl_e[:, 2] - lats_dbl_e[:, 1])


## x-index Array

In [7]:
tile_x_min_max = [
    numpy.array([0, 35]),
    numpy.array([30, 65]),
    numpy.array([60, 95]),
    numpy.array([90, 125]),
    numpy.array([120, 155]),
    numpy.array([150, 185]),
    numpy.array([180, 215]),
    numpy.array([210, 245]),
    numpy.array([240, 275]),
    numpy.array([270, 305]),
    numpy.array([300, 335]),
    numpy.array([330, 365]),
    numpy.array([360, 398]),
]

In [17]:
bathy.close()
dbl_bathy_base.close()
dbl_bathy.close()